# Trabajo Práctico N. ° 1: Análisis exploratorio de datos de TROCAFONE

Primero se cargarán los datos propuestos por la cátedra y se hará un análisis inicial del dataframe.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

events = pd.read_csv('./data/events.csv')

pd.set_option('display.max_columns', 51)

In [2]:
# analizemos primero las dimensiones del data frame
events.shape

(1011288, 23)

### Transformación de object a timestamp
Para poder trabajar con las fechas y hacer un análisis en base a esto debemos transformar los object a tipo de dato fecha.


In [3]:
events['timestamp'] = pd.to_datetime(events['timestamp'])

Agregamos las columnas de año y mes

In [4]:
# Vemos algunos datos por las fechas
events_count_per_week = events['week_year'].value_counts()
# print(events['month'].value_counts()) Tenemos valores de enero a julio
# print(events['year'].value_counts()) Tenemos valores solo de 2018
# print(events['month_day'].value_counts())
# print(events['day_week'].value_counts())
events_count_per_week.sort_values(axis=0, ascending=True)
events_count_per_week = events_count_per_week.to_frame()
events_count_per_week['week'] = events_count_per_week.index
# print(events_count_per_week) Sin ordenar aun
# print(events['year_day'].value_counts())
# Ordenamos por semana del anio
events_count_per_week.sort_values(by='week', inplace=True)
print(events_count_per_week)


KeyError: 'week_year'

In [ ]:
# Sacamos los datos de la ultima semana ya que posiblemente no tengamos informacion de la semana completa
events_count_per_week = events_count_per_week.loc[events_count_per_week['week'] != 24]
# print(events_count_per_week)

Como podemos observar tenemos solo datos del año 2018 desde enero hasta junio. Tambien podemos ver que los usuarios suelen acceder a mas a TROCAFONE los dias de semana. 
A continuacion podemos ver como aumento la frecuencia de eventos a medida que avanzo el anio

In [ ]:
plot_events_per_week = events_count_per_week[['week_year']].plot.line()
plot_events_per_week.set_title("Cantidad de eventos segun la semana del anio", fontsize=18)
plot_events_per_week.set_xlabel("Semana")
plot_events_per_week.set_ylabel("Cantidad")


In [ ]:
events['device_type'].value_counts()

In [ ]:
# Seleccionamos los eventos de sitio visitado
visited_site = events.loc[events['event'] == 'visited site' , : ]
visited_site.info()

In [ ]:
# Vemos cuanta proporcion hay de new y returning
new_vs_returning_count = visited_site['new_vs_returning'].value_counts()
print(new_vs_returning_count)

In [ ]:
plot_new_vs_returning_counts = new_vs_returning_count.plot.bar()
plot_new_vs_returning_counts.set_title("Cantidad de usuarios que generaron un evento de new y de returning", fontsize=18)
plot_new_vs_returning_counts.set_xlabel("Tipo de visita al sitio")
plot_new_vs_returning_counts.set_ylabel("Cantidad")

In [ ]:
# Queremos ver en que tipo de dispositivo acceden por primera vez los usuarios
news_visited_site = visited_site.loc[visited_site['new_vs_returning'] == 'New' , : ]
returning_visited_site = visited_site.loc[visited_site['new_vs_returning'] == 'Returning' , : ]
news_visited_site.head()

In [ ]:
# Vemos los distintos tipos de accesos por primera vez
news_visited_site['device_type'].value_counts(normalize=True)

De aqui podemos observar que la primera vez que acceden los usuarios a trocafone es casi igual de parejo entre computadoras de escritorio y celulares.

Veamos que sucede si vemos el tipo de dispositivo con el que vuelven los usuarios a la plataforma

In [ ]:
# Vemos los distintos tipos de accesos en los eventos de visita
returning_visited_site['device_type'].value_counts(normalize=True)

**Conclusion**: de estos datos inducimos que la experiencia del usuario en la plataforma es igual de buena tanto en mobile como en desktop

In [ ]:
visited_site['country'].value_counts()

In [ ]:
len(visited_site)

In [ ]:
visited_site.dropna(axis='columns', inplace=True)

# Categorizamos las regiones geograficamente
visited_site['geo_region'] = np.nan

visited_site.loc[
    (visited_site['region'] == 'Sao Paulo') |
    (visited_site['region'] == 'Rio de Janeiro') |
    (visited_site['region'] == 'Minas Gerais') |
    (visited_site['region'] == 'Espirito Santo'), 
    'geo_region'
] = 'southeast'

visited_site.loc[
    (visited_site['region'] == 'Parana') |
    (visited_site['region'] == 'Rio Grande do Sul') |
    (visited_site['region'] == 'Santa Catarina'), 
    'geo_region'
] = 'south'

visited_site.loc[
    (visited_site['region'] == 'Federal District') |
    (visited_site['region'] == 'Goias') |
    (visited_site['region'] == 'Mato Grosso do Sul') |
    (visited_site['region'] == 'Mato Grosso'), 
    'geo_region'
] = 'center west'

visited_site.loc[
    (visited_site['region'] == 'Bahia') |
    (visited_site['region'] == 'Pernambuco') |
    (visited_site['region'] == 'Ceara') |
    (visited_site['region'] == 'Maranhao') |
    (visited_site['region'] == 'Rio Grande do Norte') |
    (visited_site['region'] == 'Paraíba') |
    (visited_site['region'] == 'Piaui') |
    (visited_site['region'] == 'Alagoas') |
    (visited_site['region'] == 'Sergipe'), 
    'geo_region'
] = 'northeast'

visited_site.loc[
    (visited_site['region'] == 'Para') |
    (visited_site['region'] == 'Amazonas') |
    (visited_site['region'] == 'Tocantins') |
    (visited_site['region'] == 'Amapa') |
    (visited_site['region'] == 'Rondonia') |
    (visited_site['region'] == 'Acre') |
    (visited_site['region'] == 'Roraima'), 
    'geo_region'
] = 'north'

In [ ]:
visited_site_brazil = visited_site.loc[visited_site['country'] == 'Brazil']


cities = visited_site_brazil['city'].value_counts()
regions = visited_site_brazil['region'].value_counts()
countries = visited_site_brazil['country'].value_counts()
geo_region = visited_site_brazil['geo_region'].value_counts(normalize=True)
# print regions
# print countries
print geo_region
visited_site_brazil['geo_region'].count()


In [ ]:
bar_regions_plot = regions.plot.bar(figsize=(20, 15), fontsize=40)
bar_regions_plot.set_title("Cantidad de eventos segun la region de Brasil", fontsize=70)
bar_regions_plot.set_xlabel("Region de Brasil", fontsize=50)
bar_regions_plot.set_ylabel("Cantidad", fontsize=50)
bar_regions_plot.get_figure().savefig('visited_per_region')

In [ ]:
bar_geo_regions_plot = geo_region.plot.bar(figsize=(40, 25), fontsize=40, color=['#CC0000', '#1D97E9', '#DBC742', '#990099', '#00994C'])
bar_geo_regions_plot.set_title("Cantidad de eventos segun la region geografica de Brasil", fontsize=70)
bar_geo_regions_plot.set_xlabel("Region geografica de Brasil", fontsize=50)
bar_geo_regions_plot.set_ylabel("Cantidad", fontsize=50)
bar_geo_regions_plot.get_figure().savefig('visited_per_geo_region')

In [ ]:
# Distintos tipos de dispositivos
visited_site_brazil['device_type'].value_counts()

In [ ]:
def reduceColumnLevel(df):
    l0 = df.columns.get_level_values(0)
    l1 = df.columns.get_level_values(1)
    df.columns = l0 + '_' + l1
    return df

In [ ]:
visited_site_brazil_computer_or_smarthpone = visited_site_brazil.loc[(visited_site_brazil['device_type'] == 'Computer') | (visited_site_brazil['device_type'] == 'Smartphone')]

In [ ]:
new_per_dispositive = visited_site_brazil_computer_or_smarthpone.loc[visited_site_brazil_computer_or_smarthpone['new_vs_returning'] == 'New']

In [ ]:
count_new_dispositive = new_per_dispositive['device_type'].value_counts(normalize=True)
plot_count_new_dispositive = count_new_dispositive.plot.bar(figsize=(10, 10), fontsize=15, color=['b', 'g'])
plot_count_new_dispositive.set_title("Cantidad de usuarios que entraron por primera vez al sitio segun su dispositivo", fontsize=20)
plot_count_new_dispositive.set_xlabel("Dispositivo", fontsize=20)
plot_count_new_dispositive.set_ylabel("Cantidad", fontsize=20)

In [ ]:
returning_per_dispositive = visited_site_brazil_computer_or_smarthpone.loc[visited_site_brazil_computer_or_smarthpone['new_vs_returning'] == 'Returning']

In [ ]:
returning_per_dispositive['device_type'].value_counts(normalize=True)
count_ret_dispositive = returning_per_dispositive['device_type'].value_counts(normalize=True)
plot_count_ret_dispositive = count_ret_dispositive.plot.bar(figsize=(10, 10), fontsize=15, color=['b', 'g'])
plot_count_ret_dispositive.set_title("Cantidad de usuarios que regresaron al sitio segun su dispositivo", fontsize=20)
plot_count_ret_dispositive.set_xlabel("Dispositivo", fontsize=20)
plot_count_ret_dispositive.set_ylabel("Cantidad", fontsize=20)

In [ ]:
devices_per_region = visited_site_brazil_computer_or_smarthpone.groupby(['geo_region', 'device_type']).agg({'event': ['count']})
devices_per_region.head(20)

In [ ]:
devices_per_region = reduceColumnLevel(devices_per_region)
devices_per_region.head(20)

In [ ]:
devices_per_region.unstack()

In [ ]:
devices_per_region_pivot = devices_per_region.pivot_table(index='geo_region', columns='device_type', values='event_count')
devices_per_region_pivot = devices_per_region_pivot.sort_values(by='Computer', ascending=False)
devices_per_region_pivot

In [ ]:
plot_devices_per_geo_region = devices_per_region_pivot.plot.bar(
    figsize=(10, 10), 
    fontsize=15,
    color=['g', 'b']
)
plot_devices_per_geo_region.set_title("Dispositivo segun region geografica", fontsize=20)
plot_devices_per_geo_region.set_xlabel("Dispositivo", fontsize=20)
plot_devices_per_geo_region.set_ylabel("Cantidad de accesos", fontsize=20)

In [ ]:
def filterVisitedSitesByGeoRegion(df, region):
    return df.loc[df['geo_region'] == region]

In [ ]:
def getWeekendVisites(df):
    return df.loc[df['is_weekend'] == True]

In [ ]:
def getWeekdaysVisites(df):
    return df.loc[df['is_weekend'] == False]

In [ ]:
# Accesos de los dispositivos segun la hora del dia en fin de semanas
tmp = visited_site_brazil_computer_or_smarthpone
visited_site_not_weekend = getWeekdaysVisites(tmp)
visited_site_weekend = getWeekendVisites(tmp)
devices_per_hour_on_weekend = visited_site_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})
devices_per_hour_on_weekend.head(10)

In [ ]:
devices_per_hour_not_weekend = visited_site_not_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})
devices_per_hour_not_weekend.head(0)
# devices_per_hour_not_weekend.index

In [ ]:
def plotEventCountByDevicePerHour(df, figsize=(8,8), fontsize=20):
    l0 = df.columns.get_level_values(0)
    l1 = df.columns.get_level_values(1)
    df.columns = l0 + '_' + l1
    unstacked = df.unstack()
    l0 = unstacked.columns.get_level_values(0)
    l1 = unstacked.columns.get_level_values(1)
    unstacked.columns = l0 + '_' + l1
    df_new = pd.DataFrame({
        'Computer': unstacked['event_count_Computer'],
        'Smartphone': unstacked['event_count_Smartphone'],
        }, 
        index=unstacked.index)
    plot = df_new.plot.line(figsize=figsize, fontsize=fontsize, color=['g', 'b'])
    # plot.legend(prop={'size': 5})
    return plot

In [ ]:
plot_on_weekdays = plotEventCountByDevicePerHour(devices_per_hour_not_weekend, figsize=(20,20))
plot_on_weekdays.set_xlim(0, 23)
plot_on_weekdays.set_xlabel('Hora', fontsize=30)
plot_on_weekdays.set_ylabel('Cantidad de visitas', fontsize=30)
plot_on_weekdays.set_title('Cantidad de visitas segun el dispositivo en cada hora del dia (lunes a viernes)', fontsize=30)

In [ ]:
plot_on_weekend = plotEventCountByDevicePerHour(devices_per_hour_on_weekend)
plot_on_weekend.set_xlim(0, 23)
plot_on_weekend.set_xlabel('Hora', fontsize=30)
plot_on_weekend.set_ylabel('Cantidad de visitas', fontsize=30)
plot_on_weekend.set_title('Cantidad de visitas segun el dispositivo en cada hora del dia (fin de semana)', fontsize=30)

# Analisis de visitas segun horario particularizando en region
Ahora analizaremos las visitas segun el horario para los distintas regiones geograficas que se definieron previamente.

```
                        event_count
geo_region	device_type	

center west	Computer	2526
            Smartphone	1828
            
north	    Computer	1632
            Smartphone	1476
            
northeast	Computer	7766
            Smartphone	8619
            
south	    Computer	3737
            Smartphone	2857
            
southeast	Computer	18884
            Smartphone	21386
```

In [ ]:
tmp = visited_site_brazil_computer_or_smarthpone

center_west = filterVisitedSitesByGeoRegion(tmp, 'center west')
north = filterVisitedSitesByGeoRegion(tmp, 'north')
northeast = filterVisitedSitesByGeoRegion(tmp, 'northeast')
south = filterVisitedSitesByGeoRegion(tmp, 'south')
southeast = filterVisitedSitesByGeoRegion(tmp, 'southeast')

# Filtamos por dia de seamana y fin de semana
center_west_weekdays = getWeekdaysVisites(center_west)
center_west_weekend = getWeekendVisites(center_west)

north_weekdays = getWeekdaysVisites(north)
north_weekend = getWeekendVisites(north)

northeast_weekdays = getWeekdaysVisites(northeast)
northeast_weekend = getWeekendVisites(northeast)

south_weekdays = getWeekdaysVisites(center_west)
south_weekend = getWeekendVisites(center_west)

southeast_weekdays = getWeekdaysVisites(southeast)
southeast_weekend = getWeekendVisites(southeast)

# Agrupamos
cw_group_wd = center_west_weekdays.groupby(['hour', 'device_type']).agg({'event': ['count']})
cw_group_we = center_west_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})

n_group_wd = north_weekdays.groupby(['hour', 'device_type']).agg({'event': ['count']})
n_group_we = north_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})

ne_group_wd = northeast_weekdays.groupby(['hour', 'device_type']).agg({'event': ['count']})
ne_group_we = northeast_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})

s_group_wd = south_weekdays.groupby(['hour', 'device_type']).agg({'event': ['count']})
s_group_we = south_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})

se_group_wd = southeast_weekdays.groupby(['hour', 'device_type']).agg({'event': ['count']})
se_group_we = southeast_weekend.groupby(['hour', 'device_type']).agg({'event': ['count']})



In [ ]:
# Plot center west
p_cw_wd = plotEventCountByDevicePerHour(cw_group_wd)
p_cw_wd.set_xlim(0,23)
p_cw_wd.set_xlabel('Hora', fontsize=30)
p_cw_wd.set_ylabel('Cantidad de visitas', fontsize=30)
p_cw_wd.set_title('Dias de semana en el centro oeste', fontsize=30)

p_cw_we = plotEventCountByDevicePerHour(cw_group_we)
p_cw_we.set_xlim(0,23)
p_cw_we.set_xlabel('Hora', fontsize=30)
p_cw_we.set_ylabel('Cantidad de visitas', fontsize=30)
p_cw_we.set_title('Fin de semana en el centro oeste', fontsize=30)

In [ ]:
# Plot north
p_n_wd = plotEventCountByDevicePerHour(n_group_wd)
p_n_wd.set_xlim(0,23)
p_n_wd.set_xlabel('Hora', fontsize=30)
p_n_wd.set_ylabel('Cantidad de visitas', fontsize=30)
p_n_wd.set_title('Dias de semana en el norte', fontsize=30)

p_n_we = plotEventCountByDevicePerHour(n_group_we)
p_n_we.set_xlim(0,23)
p_n_we.set_xlabel('Hora', fontsize=30)
p_n_we.set_ylabel('Cantidad de visitas', fontsize=30)
p_n_we.set_title('Fin de semana en el norte', fontsize=30)

In [ ]:
# Plot northeast
p_ne_wd = plotEventCountByDevicePerHour(ne_group_wd)
p_ne_wd.set_xlim(0,23)
p_ne_wd.set_xlabel('Hora', fontsize=30)
p_ne_wd.set_ylabel('Cantidad de visitas', fontsize=30)
p_ne_wd.set_title('Dias de semana en el noreste', fontsize=30)

p_ne_we = plotEventCountByDevicePerHour(ne_group_we)
p_ne_we.set_xlim(0,23)
p_ne_we.set_xlabel('Hora', fontsize=30)
p_ne_we.set_ylabel('Cantidad de visitas', fontsize=30)
p_ne_we.set_title('Fin de semana en el noreste', fontsize=30)

In [ ]:
# Plot south
p_s_wd = plotEventCountByDevicePerHour(s_group_wd)
p_s_wd.set_xlim(0,23)
p_s_wd.set_xlabel('Hora', fontsize=30)
p_s_wd.set_ylabel('Cantidad de visitas', fontsize=30)
p_s_wd.set_title('Dias de semana en el sur', fontsize=30)

p_s_we = plotEventCountByDevicePerHour(s_group_we)
p_s_we.set_xlim(0,23)
p_s_we.set_xlabel('Hora', fontsize=30)
p_s_we.set_ylabel('Cantidad de visitas', fontsize=30)
p_s_we.set_title('Fin de semana en el sur', fontsize=30)

In [ ]:
# Plot southeast
p_se_wd = plotEventCountByDevicePerHour(se_group_wd)
p_se_wd.set_xlim(0,23)
p_se_wd.set_xlabel('Hora', fontsize=30)
p_se_wd.set_ylabel('Cantidad de visitas', fontsize=30)
p_se_wd.set_title('Dias de semana en el sureste', fontsize=30)

p_se_we = plotEventCountByDevicePerHour(se_group_we)
p_se_we.set_xlim(0,23)
p_se_we.set_xlabel('Hora', fontsize=30)
p_se_we.set_ylabel('Cantidad de visitas', fontsize=30)
p_se_we.set_title('Fin de semana en el sureste', fontsize=30)